In [ ]:
import pandas as pd
import altair as alt

In [ ]:
# DB original

df = pd.read_csv('/content/magdalena_database_limpia_01.csv', sep=';')
engine='python'

df = df.dropna(how='all')

print(df.to_string())

      Año         Mes  Buzo  Polerón  Zapatillas  Top  Falda  Blusa  Zapatos  Bláiser IPC Vestuario y Calzado Variación Mensual (%)
0    2014       Enero     7        5          12   23     11      3       15        1                  121,73                  -2,8
1    2014     Febrero    10        8          14   23     12      3       18        1                  122,94                     1
2    2014       Marzo     9       11          16   20     10      3       20        1                  124,61                   1,4
3    2014       Abril     8       11          15   21     12      2       20        1                  124,44                  -0,1
4    2014        Mayo    10       11          14   22     12      3       22        1                  123,69                  -0,6
5    2014       Junio    10       11          13   20     10      2       20        1                  121,19                    -2
6    2014       Julio     9       10          15   24     11      3       19

In [ ]:
import pandas as pd


df = pd.read_csv('magdalena_database_limpia_01.csv', sep=';')


low_cost_cols = ['Buzo', 'Polerón', 'Zapatillas', 'Top']
high_cost_cols = ['Falda', 'Bláiser', 'Zapatos', 'Blusa']

low_cost_mean = df[low_cost_cols].mean()
high_cost_mean = df[high_cost_cols].mean()

print("Promedios de interés por categoría:")
print(low_cost_mean.to_string())
print(high_cost_mean.to_string())


top_low = low_cost_mean.idxmax()
top_high = high_cost_mean.idxmax()

print(f"Mayor interés low cost: {top_low}")
print(f"Mayor interés high cost: {top_high}")

#DB con reconocimiento de prendas de más interés
df_final = df[['Año', 'Mes', top_low, top_high, 'IPC Vestuario y Calzado', 'Variación Mensual (%)']].copy()

df_final.rename(columns={
    top_low: 'Interés Low Cost',
    top_high: 'Interés High Cost'
}, inplace=True)

Promedios de interés por categoría:
Buzo          16.704545
Polerón       32.750000
Zapatillas    43.151515
Top           17.833333
Falda      18.181818
Bláiser     2.363636
Zapatos    25.500000
Blusa       6.590909
Mayor interés low cost: Zapatillas
Mayor interés high cost: Zapatos


In [20]:
df_final['Fecha'] = df_final['Año'] + '-' + df_final['Mes']
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], format='%Y-%B', errors='coerce')
df_final = df_final.set_index('Fecha')
display(df_final)

,Año,Mes,Interés Low Cost,Interés High Cost,IPC Vestuario y Calzado,Variación Mensual (%)
Fecha,,,,,,
NaT,2014,Enero,12.0,15.0,"121,73","-2,8"
NaT,2014,Febrero,14.0,18.0,"122,94","1,0"
NaT,2014,Marzo,16.0,20.0,"124,61","1,4"
NaT,2014,Abril,15.0,20.0,"124,44","-0,1"
NaT,2014,Mayo,14.0,22.0,"123,69","-0,6"
...,...,...,...,...,...,...
NaT,2024,Septiembre,83.0,29.0,"98,88","3,3"
NaT,2024,Octubre,74.0,28.0,"95,94","-3,0"
NaT,2024,Noviembre,75.0,29.0,"97,22","1,3"


In [ ]:
import altair as alt
import pandas as pd

df_melt = df_final.melt(
    id_vars=['Fecha'],
    value_vars=['Interés Low Cost', 'Interés High Cost', 'IPC Vestuario y Calzado'],
    var_name='Categoría',
    value_name='Valor'
)

# Gráfico de líneas de interés y IPC
grafico_final = alt.Chart(df_melt).mark_line(point=True).encode(
    x=alt.X('Fecha:T', title='Fecha'),
    y=alt.Y('Valor:Q', title='Valor'),
    color=alt.Color('Categoría:N', title='Categoría'),
    tooltip=['Fecha:T', 'Categoría', 'Valor']
).properties(
    title='Evolución del interés en moda (Low vs High Cost) y el IPC Vestuario y Calzado',
    width=800,
    height=400
).interactive()

grafico_final

alt.Chart(...)

In [ ]:

df_final['IPC Vestuario y Calzado'] = df_final['IPC Vestuario y Calzado'].str.replace(',', '.', regex=False).astype(float)

correlation_matrix = df_final[['Interés Low Cost', 'Interés High Cost', 'IPC Vestuario y Calzado']].corr()

print("Correlation Matrix:")
display(correlation_matrix)

Correlation Matrix:


,Interés Low Cost,Interés High Cost,IPC Vestuario y Calzado
Interés Low Cost,1.000000,0.690439,-0.631668
Interés High Cost,0.690439,1.000000,-0.415510
IPC Vestuario y Calzado,-0.631668,-0.415510,1.000000


In [27]:
grafico_final.save('grafico_moda.html')